The url parameter of the dl.GeoJSON component is pointing to a local file ("/home/jan/Uni/DS-Project/modules/dashboard/us-states.js"). This file will only be accessible to the server, and not to the client-side JavaScript that is running in the user's web browser. To display the polygons on the map, you will need to either:

- Host the us-states.js file on a public server and provide the URL to that server in the dl.GeoJSON component.
- Serve the us-states.js file from the same server that is running the Dash app, and provide the URL to that file in the dl.GeoJSON component.

In [1]:
import os
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html, Output, Input, dcc
from dash_extensions.javascript import arrow_function, assign
import json
import urllib.request

In [2]:
us_states_path = "assets/us-states.json"
# Download asset if it doesn't exists.
os.makedirs("assets", exist_ok=True)
if not os.path.isfile(us_states_path):
    urllib.request.urlretrieve("http://dash-leaflet.herokuapp.com/assets/us-states.json", us_states_path)
# Load us states.
with open(us_states_path) as f:
    states = [f["properties"] for f in json.load(f)["features"]]

In [3]:
states[0]

{'name': 'Alabama', 'density': 94.65}

In [3]:
def get_info(feature=None):
    header = [html.H4("US Population Density")]
    if not feature:
        return header + [html.P("Hoover over a state")]
    return header + [html.B(feature["properties"]["name"]), html.Br(),
                     "{:.3f} people / mi".format(feature["properties"]["density"]), html.Sup("2")]

classes = [0, 10, 20, 50, 100, 200, 500, 1000]
colorscale = ['#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026']
style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)
# Create colorbar.
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])] + ["{}+".format(classes[-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")
# Create geojson.
geojson = dl.GeoJSON(url=f"/{us_states_path}",  # url to geojson file
                     options=dict(style=style_handle),  # how to style each polygon
                     zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="density"),
                     id="geojson")
# Create info control.
info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10px", "right": "10px", "z-index": "1000"})
# Create app.
app = Dash(prevent_initial_callbacks=True)
app.layout = html.Div([dl.Map(children=[dl.TileLayer(), geojson, colorbar, info])],
                      style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map")


@app.callback(Output("info", "children"), [Input("geojson", "hover_feature")])
def info_hover(feature):
    return get_info(feature)

In [4]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [04/Jan/2023 11:27:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:27:59] "GET /assets/dashExtensions_default.js?m=1672828060.2624562 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:28:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 11:28:01] "GET /_dash-dependencies HTTP/1.1" 200 -


links:
- https://community.plotly.com/t/dash-leaflet-efficiency-filtering-a-large-geojson-11-mb/52785/2
- https://github.com/thedirtyfew/dash-leaflet/issues/14
- https://leafletjs.com/examples/choropleth/ [java]
- https://dash-leaflet-docs.onrender.com/#